In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import sklearn.model_selection as ms
from sklearn import ensemble
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ah = pd.read_csv('Ames_Housing_Price_Data.csv', index_col=0,low_memory=False)

<h1>01: Cleaning</h1>

In [4]:
ah = ah.fillna(0)

In [5]:
hoods = ah['Neighborhood']
new_hoods = []

for neigh in hoods :
    if neigh == 'Blueste' :
        new_hoods.append('SW')
    elif neigh == 'Blmngtn' :
        new_hoods.append('NO')
    elif neigh == 'BrDale' :
        new_hoods.append('NO')
    elif neigh == 'BrkSide' :
        new_hoods.append('DT')
    elif neigh == 'ClearCr' :
        new_hoods.append('NO')
    elif neigh == 'CollgCr' :
        new_hoods.append('SW')
    elif neigh == 'Crawfor' :
        new_hoods.append('SW')
    elif neigh == 'Edwards' :
        new_hoods.append('SW')
    elif neigh == 'Gilbert' :
        new_hoods.append('NO')
    elif neigh == 'IDOTRR' :
        new_hoods.append('DT')
    elif neigh == 'MeadowV' :
        new_hoods.append('SE')
    elif neigh == 'Mitchel' :
        new_hoods.append('SE')
    elif neigh == 'NAmes' :
        new_hoods.append('NO')
    elif neigh == 'NoRidge' :
        new_hoods.append('NW')
    elif neigh == 'NPkVill' :
        new_hoods.append('NO')
    elif neigh == 'NridgHt' :
        new_hoods.append('NW')
    elif neigh == 'NWAmes' :
        new_hoods.append('NO')
    elif neigh == 'OldTown' :
        new_hoods.append('DT')
    elif neigh == 'SWISU' :
        new_hoods.append('SW')
    elif neigh == 'Sawyer' :
        new_hoods.append('NW')
    elif neigh == 'SawyerW' :
        new_hoods.append('NW')
    elif neigh == 'Somerst' :
        new_hoods.append('NW')
    elif neigh == 'StoneBr' :
        new_hoods.append('NO')
    elif neigh == 'Timber' :
        new_hoods.append('SW')
    elif neigh == 'Veenker' :
        new_hoods.append('NW')
    elif neigh == 'Greens' :
        new_hoods.append('NW')
    elif neigh == 'GrnHill' :
        new_hoods.append('SO')
    elif neigh == 'Landmrk' :
        new_hoods.append('DT')
    else : print('no match')

ah['city_sec'] = new_hoods 

In [6]:
df_float = ah.select_dtypes(include='float')
df_float_to_int = df_float.astype(int)

<h5>❤️ Divide data into categorical and numerical, then check if those columns are equal</h5>

In [7]:
df_numerical = ah.select_dtypes(include='int64')
df_numerical = pd.concat([df_numerical,df_float_to_int],axis=1)
df_categorical = ah.select_dtypes(include='object')
print('There are',len(ah.columns) - (len(df_categorical.columns) + len(df_numerical.columns)),'missing columns')
if len(df_categorical.index) == len(df_numerical.index):
    print('There is an equal number of rows')
else : print('Unequal rows')
#82 columns total

There are 0 missing columns
There is an equal number of rows


In [29]:
dummies = pd.get_dummies(df_categorical,drop_first=True)
x = pd.concat([df_numerical,dummies],axis=1)
x = x.drop(columns = ['SalePrice', 'PID'])
#x.drop('PID', axis=1, inplace=True)
y = df_numerical['SalePrice']
# x = x.drop(columns=['SalePrice','1stFlrSF','2ndFlrSF','LowQualFinSF','BsmtFinSF1','BsmtFinSF2','MSZoning_I (all)',
#                     'Neighborhood_Blueste','Exterior2nd_PreCast','BsmtQual_Po','Electrical_SBrkr','MiscFeature_TenC','city_sec_SO',
#                    'GarageQual_Po','BsmtCond_Fa','GarageCond_Fa','Neighborhood_Greens','BsmtFinType2_BLQ','city_sec_NW',
#                    'Neighborhood_MeadowV','city_sec_SE','SaleCondition_Partial','city_sec_NO'])


<h1>02: Multiple Linear Regression</h1>

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)#, random_state = 3)
mlr = LinearRegression()

In [31]:
mlr.fit(x_train,y_train)
y_predict =  mlr.predict(x_test)
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_predict})
mlr_diff

,Actual value,Predicted value
233,212300,2.146522e+05
708,153000,1.662619e+05
659,180000,1.855819e+05
658,123000,1.234327e+05
952,336820,3.093732e+05
...,...,...
677,136905,-1.188147e+09
687,228000,2.178642e+05
167,96000,9.470791e+04
251,157000,1.555042e+05


In [27]:
meanAbErr = metrics.mean_absolute_error(y_test, y_predict)
meanSqErr = metrics.mean_squared_error(y_test, y_predict)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_predict))
print('R squared: {:.2f}'.format(mlr.score(x,y) * 100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 93.42
Mean Absolute Error: 14949.17211863022
Mean Square Error: 497370518.9670404
Root Mean Square Error: 22301.805284932438


In [12]:
#vif = pd.DataFrame()
#vif['var'] = x_train.columns
#vif['VIF'] = [variance_inflation_factor(x_train.values, i) for i in range(x_train.shape[1])]

In [13]:
inf = vif['VIF'] > 5
vifs = vif.loc[inf]
vifs['var']

NameError: name 'vif' is not defined

In [28]:
x_train

,PID,GrLivArea,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,city_sec_NO,city_sec_NW,city_sec_SE,city_sec_SO,city_sec_SW
586,923400040,1750,50,12508,6,7,1940,1985,983,767,...,0,0,0,1,0,0,0,1,0,0
427,909100080,1296,30,4853,5,6,1924,1999,1296,0,...,0,0,0,1,0,0,0,0,0,1
117,534401120,1200,20,8800,7,6,1966,1966,1200,0,...,0,0,0,1,0,1,0,0,0,0
52,533206070,1565,120,3784,8,5,2006,2007,1565,0,...,0,0,0,0,1,0,1,0,0,0
527,526351010,1329,20,14267,6,6,1958,1958,1329,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,923275010,1178,20,11000,5,6,1976,2003,1178,0,...,0,0,0,1,0,0,0,1,0,0
241,528290010,1490,60,15384,7,5,2004,2005,788,702,...,0,0,0,1,0,1,0,0,0,0
780,535106190,2274,90,11600,5,5,1960,1960,1105,1169,...,0,0,0,1,0,1,0,0,0,0
422,527455270,892,20,9477,5,5,1966,1966,892,0,...,0,0,0,1,0,1,0,0,0,0


In [19]:
x_train.drop(columns='PID')

,GrLivArea,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,city_sec_NO,city_sec_NW,city_sec_SE,city_sec_SO,city_sec_SW
603,1912,190,10800,6,7,1905,2000,1221,691,0,...,0,0,0,1,0,0,0,0,0,0
403,804,30,7200,5,7,1940,1992,804,0,0,...,0,0,0,0,0,0,0,0,0,0
96,1478,120,7380,8,5,1998,1998,1478,0,0,...,0,0,0,1,0,1,0,0,0,0
8,4316,60,21535,10,6,1994,1995,2444,1872,0,...,0,0,0,1,0,0,1,0,0,0
802,1487,50,8741,6,4,1945,1950,798,689,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,1181,85,10012,4,5,1972,1972,1181,0,0,...,0,0,0,1,0,0,0,0,0,1
773,1479,20,9382,7,5,1999,2000,1479,0,0,...,0,0,0,1,0,0,0,0,0,1
232,1729,60,8158,7,5,2002,2002,860,869,0,...,0,0,0,1,0,0,0,0,0,1
126,1494,20,9236,6,5,1997,1997,1494,0,0,...,0,0,0,1,0,0,0,0,0,1


In [32]:
y_train

472    229000
315    415000
422    213000
747    119000
4      183000
        ...  
67     171500
117    159500
490    262500
57     176000
506    127500
Name: SalePrice, Length: 2064, dtype: int64